# Проект по SQL-запросам

Компания купила крупный сервис для чтения книг по подписке и хочет сформулировать ценовое предложение для нового продукта.

Чтобы помочь с этим, необходимо проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательскиx обзорах книг, выполнив несколько SQL-запросов.

## Подключение к базе данных 

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Изучение таблиц

In [3]:
#напишем функцию, которая будет выводить запрос на экран:
def query(query):
    result = pd.io.sql.read_sql(query, con = engine)
    return result

In [4]:
books = '''
SELECT *
FROM books
LIMIT 10'''

query(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


Это таблица с информацией о книгах - названии, количеством страниц, датой публикацией, а также идентификаторам издательства и автора, отсылающим к соответствующим таблицам.

In [5]:
authors = '''
SELECT *
FROM authors
LIMIT 10'''

query(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


В этой таблице содержатся имена авторов книг.

In [6]:
publishers = """
SELECT *
FROM publishers
LIMIT 10"""

query(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


В этой таблице - названия издательств.

In [7]:
ratings = """
SELECT *
FROM ratings
LIMIT 10"""

query(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


В данной таблице представлены оценки пользователей для разных книг. Здесь есть поля с никнеймом пользователя, поставленной им оценкой, а также идентификатором книги, которой была поставлена оценка.

In [8]:
reviews = """
SELECT *
FROM reviews
LIMIT 10"""

query(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


Это таблица с обзорами. Здесь есть поля с никнейном пользователя и текстом его ообзора, а также идентификатор книги, о которой был написан обзор.

## Выполнение SQL-запросов

**Задание 1. Посчитайте, сколько книг вышло после 1 января 2000 года.**

In [9]:
query1 = '''
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01';
'''

query(query1)

,count
0,819


После 1 января 2020 года (не включительно) вышло 819 книг.


**Задание 2. Для каждой книги посчитайте количество обзоров и среднюю оценку.**

In [10]:
query2 = '''
SELECT books.book_id,
books.title,
COUNT(DISTINCT reviews.review_id),
AVG(ratings.rating)
FROM books
LEFT JOIN ratings ON books.book_id=ratings.book_id
LEFT JOIN reviews ON books.book_id=reviews.book_id
GROUP BY 1, 2
ORDER BY count DESC
'''

query(query2)

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


Больше всего обзоров у следующих книг:
- Twilight: 7 обзоров, средний рейтинг - 3.66
- Water for Elephants: 6 обзоров, средний рейтинг 3.98
- The Glass Castle: 6 обзоров, средний рейтинг 4.21

**Задание 3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.**

In [11]:
query3 = '''
SELECT publisher,
COUNT(books.book_id)
FROM publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE num_pages > 50
GROUP BY publisher
ORDER BY COUNT(books.book_id) DESC
'''

query(query3)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Больше всего книг толще 50 страниц - 42 штуки - выпустило издательство Penguin Books.


**Задание 4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками**

In [13]:
query4 = '''
WITH a AS (SELECT authors.author,
                  books.book_id,
                  AVG(ratings.rating),
                  COUNT(ratings.rating)
           FROM authors
           INNER JOIN books ON books.author_id = authors.author_id
           INNER JOIN ratings ON ratings.book_id = books.book_id
           GROUP BY author,
                    books.book_id
           HAVING COUNT(ratings.rating) >= 50)

SELECT  a.author,
        AVG(a.avg)
FROM a
GROUP BY author
ORDER BY avg DESC
'''

query(query4)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самая высокая средняя оценка книг у Джоан Роулинг - 4,28 балла.

**Задание 5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.**

In [ ]:
query5 = '''
WITH a AS
(SELECT
COUNT(review_id) AS cnt
FROM reviews
WHERE username IN(SELECT username
                  FROM ratings
                  GROUP BY username
                  HAVING COUNT(rating_id) > 50)
GROUP BY username)

SELECT
AVG(a.cnt)
FROM a
'''

query(query5)

Пользователи, поставившие больше 50 оценок, пишут в среднем около 24 обзоров на книги.

## Выводы

- После 1 января 2020 года (не включительно) вышло 819 книг.
- Больше всего обзоров у следующих книг:
    - Twilight: 7 обзоров, средний рейтинг - 3.66
    - Water for Elephants: 6 обзоров, средний рейтинг 3.98
    - The Glass Castle: 6 обзоров, средний рейтинг 4.21
- Больше всего книг толще 50 страниц - 42 штуки - выпустило издательство Penguin Books.
- Самая высокая средняя оценка книг у Джоан Роулинг - 4,28 балла.
- Пользователи, поставившие больше 50 оценок, пишут в среднем около 24 обзоров на книги.